In [160]:
import io
import re

import string
import tempfile 


from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

In [159]:
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.layout import LAParams

from pdfminer.high_level import extract_text_to_fp

from arabic_reshaper import ArabicReshaper
from bidi.algorithm import get_display

In [11]:
configuration = {
    'delete_harakat': True,
    'delete_tatweel ': True,
}

year_folder_id_dict = {'2010':'113DAB_g6P3b-gbrpFog682EfdILcBMt6','2011':'1Yh9610KUMxZLNDiyUOnng-7PbhlYbt3x',
                       '2012':'1TEWAyyxf6dJ3K6hMiliayJ4VV4ilvwIH','2013':'1KWNg6Me11spV-JRFOzwlvNpvoYMLI59C',
                       '2014':'1IgDuaPlc9b-MdP6VELFXcTffqSRHYPU5','2015':'1CrHueAEH76p95tIOryMrdu5nV-qW702M',
                       '2016':'1hpViY56I20A6k6EeQGrOn6QG11j7gb58','2017':'1H2HXwALMyzxBIMJyluYkTHJKplqGIpUF',
                       '2018':'1-jlTY-u_izedKKNenevC3kgM6yaKR7b7','2019':'1Qm-Sse4vbOx2IOAILUCTvXwqShWlynQn',}

In [149]:
def convert_text(word):
    regex = r"({.*?})"
    word = re.findall(regex,str(word.encode('WINDOWS-1251',errors='namereplace')))
    cleaned = []
    for char in word:
        char = char.replace('{','').replace('}','')
        cleaned.append(char)
    
    cleaned_word = []
    for x in cleaned:
       if len(x.split()) > 2:
        cleaned_word.append(' '.join(i for i in x.split()[:3]))
        
    return cleaned_word

In [152]:
section_keyword = ['لبنان','اقتصاد','مجتمع','سياسة']
section_keyword_encoded = [convert_text(x) for x in section_keyword]

In [162]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)
folder_id = year_folder_id_dict['2016']

In [163]:
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
files = results.get('files', [])

# Iterate through the files
for file in files:
    file_id = file.get('id')
    file_name = file.get('name')
    
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        #print("Download %d%%." % int(status.progress() * 100))
    pdf_content = fh.getvalue()
    with open('new.pdf', 'wb') as f:
        f.write(pdf_content)
    break

In [164]:
pdf = PdfFileReader(open('new.pdf', 'rb'))

texts = ''

for page in range(1,pdf._get_num_pages()):
    with open('new.pdf', 'rb') as f:
        laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
        outfp = io.StringIO()
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        corrected_text = get_display(text)
        reshaped_text = reshaper.reshape(corrected_text)
        
        
        section = ''.join(x for x in reshaped_text[:60] if x not in string.digits).strip().replace('\n',' ').replace('ـ', '')
        section = [x for x in section.split() if len(x) > 2]
        section_encoded = [convert_text(x) for x in section]
        
        if any(x in section_keyword_encoded for x in section_encoded):
            
            output_pdf = PdfFileWriter()
            output_pdf.addPage(pdf.getPage(page))
            name = file_name.replace('.pdf', f'-page-{int(page)+1}.pdf')
            with open(name, 'wb') as f:
                output_pdf.write(f)
                file = tempfile.NamedTemporaryFile()
                file.write(open(name, 'rb').read())
                file.seek(0)
                file_metadata = {'name': name,'parents':['1CPmsZoNqsDslYUDuMMYAiTRdgcbjuIj6']}
                
                media = MediaFileUpload(file.name,
                                mimetype='application/pdf')
                
                service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    

In [143]:
cleaned_text = [convert_text(x) for x in texts.split()]